In [7]:
import torchvision
print(torchvision.__version__)

0.21.0+cu124


In [1]:
from utils import Config
from data_loader import MultiOmicsDataset, create_dataloaders

In [26]:
# # Reload all modules every time before executing code
%load_ext autoreload
%autoreload 2  

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 1. Test Config class

In [3]:
config = Config.from_yaml("configs/data_config.yaml")
config

Config({'data_dir': './data/clean_data', 'omics': Config({'methyl': Config({'file': 'methy.csv', 'dtype': 'float32', 'reshape': [50, 100]}), 'mirna': Config({'file': 'mirna.csv', 'dtype': 'float32'}), 'rna': Config({'file': 'exp.csv', 'dtype': 'float32'})}), 'labels': Config({'file': 'label.csv', 'dtype': 'int16', 'squeeze': True}), 'loader': Config({'batch_size': 64, 'num_workers': 8, 'seed': 42, 'splits': Config({'train': 0.7, 'val': 0.15, 'test': 0.15})})})

In [4]:
dataset = MultiOmicsDataset(config)

In [5]:
[(name, x.shape) for name, x in dataset.data.items()]

[('methyl', (439, 50, 100)), ('mirna', (439, 1046)), ('rna', (439, 13054))]

In [6]:
config_mirna = Config.from_yaml("configs/data_config_mirna.yaml")
config_mirna

Config({'data_dir': './data/clean_data', 'omics': Config({'mirna': Config({'file': 'mirna.csv', 'dtype': 'float32'})}), 'labels': Config({'file': 'label.csv', 'dtype': 'int16', 'squeeze': True})})

In [7]:
dataset_mirna  =  MultiOmicsDataset(config_mirna)

In [8]:
dataset_mirna[1] 

({'mirna': tensor([13.0402, 14.0594, 13.0643,  ...,  6.0753,  9.3519, 16.5759])},
 tensor(0))

### 2. Test Dataset and dataloader

In [9]:
config = Config.from_yaml("configs/data_config_wo_reshape.yaml")

In [10]:
dataset = MultiOmicsDataset(config)

In [26]:
dataloaders = create_dataloaders(dataset, config)

In [27]:
# Check if wi_reshpae option works

config = Config.from_yaml("configs/data_config.yaml")

dataset = MultiOmicsDataset(config)
dataloaders = create_dataloaders(dataset, config)

for batch in dataloaders['train']:
    batch_data, batch_labels = batch
    
    print(batch_data['methyl'].shape) # Shape: (B, C, H, W)
    print( batch_data['rna'].shape)   

torch.Size([64, 1, 50, 100])
torch.Size([64, 13054])
torch.Size([64, 1, 50, 100])
torch.Size([64, 13054])
torch.Size([64, 1, 50, 100])
torch.Size([64, 13054])
torch.Size([64, 1, 50, 100])
torch.Size([64, 13054])
torch.Size([51, 1, 50, 100])
torch.Size([51, 13054])


In [29]:
# Check if wo_reshpae option works

config = Config.from_yaml("configs/data_config_wo_reshape.yaml")

dataset = MultiOmicsDataset(config)
dataloaders = create_dataloaders(dataset, config)

dataset = MultiOmicsDataset(config)
for batch in dataloaders['train']:
    batch_data, batch_labels = batch
    
    print(batch_data['methyl'].shape) 
    print( batch_data['rna'].shape) 

torch.Size([64, 1, 5000])
torch.Size([64, 13054])
torch.Size([64, 1, 5000])
torch.Size([64, 13054])
torch.Size([64, 1, 5000])
torch.Size([64, 13054])
torch.Size([64, 1, 5000])
torch.Size([64, 13054])
torch.Size([51, 1, 5000])
torch.Size([51, 13054])


### 3. Test VAEencoder

In [33]:

from models.vae import VAEEncoder  # adjust the import path to match your repo

# Choose modality: "mirna" or "rna"
modality = "mirna"  # ← change this as needed

# Load config and dataset
config = Config.from_yaml("configs/data_config_wo_reshape.yaml")
input_dim = 1046
latent_dim = 64

dataset = MultiOmicsDataset(config)
dataloaders = create_dataloaders(dataset, config)

# Instantiate VAE
vae = VAEEncoder(input_dim=input_dim, latent_dim=latent_dim)
vae.eval()

# Test VAE with one batch
for batch_data, batch_labels in dataloaders["train"]:
    x = batch_data[modality]
    print("Input shape:", x.shape)

    with torch.no_grad():
        z, mu, logvar = vae(x)
        kl = vae.kl_divergence(mu, logvar)

    print("✅ z shape:     ", z.shape)
    print("✅ mu shape:    ", mu.shape)
    print("✅ logvar shape:", logvar.shape)
    print("✅ KL divergence:", kl.item())

    assert z.shape == (x.size(0), latent_dim)
    assert mu.shape == logvar.shape == (x.size(0), latent_dim)

    break  # just test one batch



Input shape: torch.Size([64, 1046])
✅ z shape:      torch.Size([64, 64])
✅ mu shape:     torch.Size([64, 64])
✅ logvar shape: torch.Size([64, 64])
✅ KL divergence: nan


### 4. Test convnext model

In [29]:
import torch
from models import MiniConvNeXtMethylation  # adjust path as needed


# Load config
config = Config.from_yaml("configs/data_config.yaml")


# Load dataset and dataloaders
dataset = MultiOmicsDataset(config)
dataloaders = create_dataloaders(dataset, config)

# Instantiate encoder (output latent dim should match your model)
latent_dim = 64
model = MiniConvNeXtMethylation(latent_dim=latent_dim)
model.eval()  # optional: disable dropout, etc.

# Test one batch from training data
for batch_data, batch_labels in dataloaders["train"]:
    methyl_batch = batch_data["methyl"]  # shape: (B, 1, 50, 100)
    
    # Forward pass
    with torch.no_grad():
        out = model(methyl_batch)

    print("✅ Input shape: ", methyl_batch.shape)
    print("✅ Output shape:", out.shape)

    # Optional: assert expected output shape
    assert out.shape == (methyl_batch.size(0), latent_dim), \
        f"Expected output shape ({methyl_batch.size(0)}, {latent_dim}), but got {out.shape}"

    break  # only test one batch


✅ Input shape:  torch.Size([64, 1, 50, 100])
✅ Output shape: torch.Size([64, 64])


### 5. Test MultimodalFusion

In [35]:
import torch
from models import MultimodalFusion  # adjust path as needed

def test_multimodal_fusion():
    batch_size = 8
    latent_dim = 64

    # Simulated modality embeddings (from encoder output)
    methy = torch.randn(batch_size, latent_dim)
    mirna = torch.randn(batch_size, latent_dim)
    rna   = torch.randn(batch_size, latent_dim)

    # Instantiate fusion module
    fusion_model = MultimodalFusion(latent_dim=latent_dim, num_heads=4)
    fusion_model.eval()

    # Forward pass
    with torch.no_grad():
        fused = fusion_model([methy, mirna, rna])

    print("✅ Input shapes: methy", methy.shape, "mirna", mirna.shape, "rna", rna.shape)
    print("✅ Fused output shape:", fused.shape)

    # Assertions
    assert fused.shape == (batch_size, latent_dim), \
        f"Expected output shape ({batch_size}, {latent_dim}), but got {fused.shape}"

    print("✅ MultimodalFusion test passed!")

# Run the test
test_multimodal_fusion()


✅ Input shapes: methy torch.Size([8, 64]) mirna torch.Size([8, 64]) rna torch.Size([8, 64])
✅ Fused output shape: torch.Size([8, 64])
✅ MultimodalFusion test passed!


In [44]:

from models import MiniConvNeXtMethylation, VAEEncoder, MultimodalFusion


def test_fusion_with_real_data():
    # Load config and dataset
    config = Config.from_yaml("configs/data_config.yaml")
    dataloaders = create_dataloaders(MultiOmicsDataset(config), config)
    
    # Get model input dims
    mirna_dim = 1046
    rna_dim = 13054
    methy_shape = (50, 100)  # e.g., (50, 100)
    latent_dim = 64

    # Initialize encoders
    methy_encoder = MiniConvNeXtMethylation(latent_dim=latent_dim, input_size=methy_shape)
    mirna_encoder = VAEEncoder(input_dim=mirna_dim, latent_dim=latent_dim)
    rna_encoder = VAEEncoder(input_dim=rna_dim, latent_dim=latent_dim)

    # Fusion model
    fusion_model = MultimodalFusion(latent_dim=latent_dim, num_heads=4)

    # Get one batch from train loader
    for batch_data, batch_labels in dataloaders['train']:
        x_methy = batch_data['methyl']  # shape: [B, 1, 50, 100]
        x_mirna = batch_data['mirna']   # shape: [B, mirna_dim]
        x_rna   = batch_data['rna']     # shape: [B, rna_dim]

        # Encode each modality
        with torch.no_grad():
            z_methy = methy_encoder(x_methy)
            z_mirna, _, _ = mirna_encoder(x_mirna)
            z_rna, _, _ = rna_encoder(x_rna)

            # Fuse features
            fused = fusion_model([z_methy, z_mirna, z_rna])

        # Print and assert
        print("✅ Encoded methylation:", z_methy.shape)
        print("✅ Encoded mirna:      ", z_mirna.shape)
        print("✅ Encoded rna:        ", z_rna.shape)
        print("✅ Fused output shape: ", fused.shape)

        assert fused.shape == (x_methy.size(0), latent_dim), "Fused output shape mismatch"
        print("✅ Real-data MultimodalFusion test passed!")

        break  # only one batch

# Run the test
test_fusion_with_real_data()


✅ Encoded methylation: torch.Size([64, 64])
✅ Encoded mirna:       torch.Size([64, 64])
✅ Encoded rna:         torch.Size([64, 64])
✅ Fused output shape:  torch.Size([64, 64])
✅ Real-data MultimodalFusion test passed!


### 6. Test MultiOmicsClassifier

In [3]:
from models import MultiOmicsClassifier  # adjust import
import torch

def test_multiomics_classifier():
    # Define input sizes
    batch_size = 4
    mirna_dim = 1046
    rna_exp_dim = 13054
    methy_shape = (50, 100)  # e.g., (50, 100)
    latent_dim = 64
    num_classes = 4

    # Instantiate the model
    model = MultiOmicsClassifier(
        mirna_dim=mirna_dim,
        rna_exp_dim=rna_exp_dim,
        methy_shape=methy_shape,
        latent_dim=latent_dim,
        num_classes=num_classes
    )
    model.eval()

    # Create fake input tensors
    x_methy = torch.randn(batch_size, 1, *methy_shape)
    x_mirna = torch.randn(batch_size, mirna_dim)
    x_rna = torch.randn(batch_size, rna_exp_dim)

    # Forward pass
    with torch.no_grad():
        outputs = model(x_methy, x_mirna, x_rna)

    # Assertions
    assert outputs['logits'].shape == (batch_size, num_classes), "❌ logits shape mismatch"
    assert outputs['mu_mirna'].shape == (batch_size, latent_dim), "❌ mu_mirna shape mismatch"
    assert outputs['logvar_mirna'].shape == (batch_size, latent_dim), "❌ logvar_mirna shape mismatch"
    assert outputs['mu_rna'].shape == (batch_size, latent_dim), "❌ mu_rna shape mismatch"
    assert outputs['logvar_rna'].shape == (batch_size, latent_dim), "❌ logvar_rna shape mismatch"

    # Print confirmations
    print("✅ logits shape:", outputs['logits'].shape)
    print("✅ mu_mirna shape:", outputs['mu_mirna'].shape)
    print("✅ mu_rna shape:", outputs['mu_rna'].shape)
    print("✅ MultiOmicsClassifier test passed!")

# Run the test
test_multiomics_classifier()



✅ logits shape: torch.Size([4, 4])
✅ mu_mirna shape: torch.Size([4, 64])
✅ mu_rna shape: torch.Size([4, 64])
✅ MultiOmicsClassifier test passed!


In [51]:

def test_multiomics_classifier_with_real_data():
    # Load config and dataset
    config = Config.from_yaml("configs/data_config.yaml")
    dataloaders = create_dataloaders(MultiOmicsDataset(config), config)

    # Get model input sizes from config
    # mirna_dim = config['omics']['mirna']['dim']
    # rna_exp_dim = config['omics']['rna']['dim']
    # methy_shape = tuple(config['omics']['methyl']['reshape'])  # e.g., (50, 100)
    # latent_dim = 64
    # num_classes = config['labels']['num_classes'] if 'num_classes' in config['labels'] else 4

    batch_size = 4
    mirna_dim = 1046
    rna_exp_dim = 13054
    methy_shape = (50, 100)  # e.g., (50, 100)
    latent_dim = 64
    num_classes = 4

    # Instantiate model
    model = MultiOmicsClassifier(
        mirna_dim=mirna_dim,
        rna_exp_dim=rna_exp_dim,
        methy_shape=methy_shape,
        latent_dim=latent_dim,
        num_classes=num_classes
    )
    model.eval()

    # Test one batch from dataloader
    for batch_data, batch_labels in dataloaders["train"]:
        x_methy = batch_data["methyl"]
        x_mirna = batch_data["mirna"]
        x_rna   = batch_data["rna"]

        with torch.no_grad():
            outputs = model(x_methy, x_mirna, x_rna)

        # Assertions
        assert outputs['logits'].shape == (x_methy.size(0), num_classes)
        assert outputs['mu_mirna'].shape == outputs['logvar_mirna'].shape == (x_methy.size(0), latent_dim)
        assert outputs['mu_rna'].shape == outputs['logvar_rna'].shape == (x_methy.size(0), latent_dim)

        # Print results
        print("✅ Real-batch MultiOmicsClassifier forward test:")
        print("   Logits shape:     ", outputs['logits'].shape)
        print("   mu_mirna shape:   ", outputs['mu_mirna'].shape)
        print("   mu_rna shape:     ", outputs['mu_rna'].shape)
        break  # just test one batch

# Run the test
test_multiomics_classifier_with_real_data()


✅ Real-batch MultiOmicsClassifier forward test:
   Logits shape:      torch.Size([64, 4])
   mu_mirna shape:    torch.Size([64, 64])
   mu_rna shape:      torch.Size([64, 64])


### 7. Run pytest

In [4]:
%%bash
pytest -v


============================= test session starts ==============================
platform linux -- Python 3.11.11, pytest-8.3.5, pluggy-1.5.0 -- /home/CBBI/wangh5/miniforge3/envs/cs7643_a4_generative/bin/python3.11
cachedir: .pytest_cache
rootdir: /home/CBBI/wangh5/_PyCharm/proj_dl
configfile: pytest.ini
plugins: anyio-4.9.0
collecting ... collected 5 items

tests/test_02_model_components.py::test_vae_forward PASSED               [ 20%]
tests/test_02_model_components.py::test_mirna_vae_forward PASSED         [ 40%]
tests/test_02_model_components.py::test_mini_convnext PASSED             [ 60%]
tests/test_02_model_components.py::test_transformer_fusion PASSED        [ 80%]
tests/test_02_model_components.py::test_fusion_classifier PASSED         [100%]

============================== 5 passed in 0.27s ===============================


In [1]:
import torch
from utils import Config
from data_loader import MultiOmicsDataset, create_dataloaders
from models import MultiOmicsClassifier 

In [2]:
from trainers import BaseTrainer
from losses import MultiOmicsLoss

In [3]:
config = Config.from_yaml("configs/data_config.yaml")

dataset = MultiOmicsDataset(config)
dataloaders = create_dataloaders(dataset, config)



In [4]:
batch_size = 4
mirna_dim = 1046
rna_exp_dim = 13054
methy_shape = (50, 100)  # e.g., (50, 100)
latent_dim = 64
num_classes = 5

# Instantiate model
multiomics_model = MultiOmicsClassifier(
    mirna_dim=mirna_dim,
    rna_exp_dim=rna_exp_dim,
    methy_shape=methy_shape,
    latent_dim=latent_dim,
    num_classes=num_classes
)

In [14]:
trainer = BaseTrainer(
    model=multiomics_model,
    optimizer=torch.optim.Adam(multiomics_model.parameters(), lr=2e-5),
    loss_fn=MultiOmicsLoss(),
    device='cuda'
)

trainer.fit(train_loader=dataloaders['train'], val_loader=dataloaders['val'], epochs=20)



Epoch 1/20


Training:   0%|                                                                                                                                                                                                                                          | 0/5 [00:00<?, ?it/s]/home/CBBI/wangh5/_PyCharm/proj_dl/losses/multi_omics.py:74: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'beta': torch.tensor(self.beta, device=self.current_step.device)
Training: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  9.40it/s, loss=0.0388]


[DEBUG] CE: 0.0330, KL_mirna: 10.3765, KL_rna: 19.4689
[DEBUG] CE: 0.0443, KL_mirna: 11.0556, KL_rna: 20.4731
[DEBUG] CE: 0.0319, KL_mirna: 11.0932, KL_rna: 20.9168
[DEBUG] CE: 0.0273, KL_mirna: 11.3598, KL_rna: 20.1461
[DEBUG] CE: 0.0377, KL_mirna: 10.9328, KL_rna: 17.5019
Train Loss: 0.0355


Validation: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  5.14it/s]


[DEBUG] CE: 0.6272, KL_mirna: 6.4601, KL_rna: 15.1710
[DEBUG] CE: 0.2024, KL_mirna: 5.2704, KL_rna: 14.2754
Val Loss:   0.4159

Epoch 2/20


...

[DEBUG] CE: 0.7433, KL_mirna: 5.8774, KL_rna: 12.5761
[DEBUG] CE: 0.8620, KL_mirna: 4.3379, KL_rna: 15.0551
Val Loss:   0.8275

Epoch 20/20


Training: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 10.10it/s, loss=0.0729]


[DEBUG] CE: 0.0231, KL_mirna: 11.2375, KL_rna: 18.7006
[DEBUG] CE: 0.0231, KL_mirna: 10.2531, KL_rna: 19.3038
[DEBUG] CE: 0.0272, KL_mirna: 10.4292, KL_rna: 18.4801
[DEBUG] CE: 0.0204, KL_mirna: 10.3263, KL_rna: 18.6598
[DEBUG] CE: 0.0340, KL_mirna: 11.1221, KL_rna: 17.2793
Train Loss: 0.0649


Validation: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  4.81it/s]

[DEBUG] CE: 0.6081, KL_mirna: 5.8226, KL_rna: 12.4856
[DEBUG] CE: 0.4399, KL_mirna: 4.2448, KL_rna: 14.9770
Val Loss:   0.5500


In [15]:
preds, targets = trainer.predict(dataloaders["test"])

# Compute accuracy
from sklearn.metrics import accuracy_score

acc = accuracy_score(targets.numpy(), preds.numpy())
print(f"✅ Test Accuracy: {acc:.4f}")


✅ Test Accuracy: 0.8182


In [16]:
from sklearn.metrics import classification_report
print(classification_report(targets.numpy(), preds.numpy()))


              precision    recall  f1-score   support

           0       0.83      0.86      0.85        29
           1       0.79      0.73      0.76        15
           2       0.90      1.00      0.95         9
           3       0.73      0.89      0.80         9
           4       1.00      0.25      0.40         4

    accuracy                           0.82        66
   macro avg       0.85      0.75      0.75        66
weighted avg       0.83      0.82      0.81        66

